Copyright (c) 2020 Martin Holle. Alle Rechte vorbehalten. Lizensiert unter der MIT-Lizenz.

# Covid-19 Statistics Aachen: Datenaufbereitung

Die rohen Fallzahlen werden zunächst aus dem im ersten Schritt erzeugten Excel-Sheet eingelesen und dann aufbereitet. Anschließend wird eine Reihe von Kennwerten berechnet. Die Ergebnisse werden in eine zweite Excel-Datei exportiert; die zwischengespeicherten Daten werden im folgenden Schritt zur Erstellung von Diagrammen verwendet.

## Benötigte Imports und Initialisierungen

In [1]:
import pandas as pd

import logging
import configparser

# Konfiguration einlesen
config = configparser.ConfigParser(interpolation=None)
config.read('config.ini')

# Konfiguration des Loggings
fhandler = logging.FileHandler(filename=config['Logging']['LogFileName'], mode='a')
formatter = logging.Formatter(config['Logging']['LogFormat'])
fhandler.setFormatter(formatter)

# Logger instanzieren
log = logging.getLogger("aufbereitung")
log.addHandler(fhandler)
log.setLevel(logging.INFO)

## Einlesen der Excel-Datei und Anzeige der eingelesenen Daten

- Datei: Siehe `config.ini`
- Seite: Siehe `config.ini`
- Einzulesende Spalten: 
  - **A**: Datum im Format 'DD.MM.'
  - **B**: Akkumulierte Anzahl der Infektionen für gesamte Städteregion (inkl. Aachen) als Integerzahl
  - **D**: Akkumulierte Anzahl der Infektionen für die Stadt Aachen als Integerzahl
  - **F**: Akkumulierte Anzahl der Todesfälle durch Covid-19 für gesamte Städteregion (inkl. Aachen) als Integerzahl 
  - **G**: Akkumulierte Anzahl der Genesenen für gesamte Städteregion (inkl. Aachen) als Integerzahl
- Spaltentypen: Spalte A als Datum interpretieren
- Die erste Zeile enthält die Spalten-Header
- Label der Spalten explizit setzen

In [2]:
col_names = ['Summe', 'Summe Aachen', 'Summe Todesfälle', 'Summe genesen', 'Akute Fälle' ]
c19_cases = pd.read_excel(config['Rohdaten']['FileName'], 
                          sheet_name=config['Rohdaten']['SheetName'],
                          usecols='A,C,D,E,F,G', 
                          index_col=0,
                          parse_dates=[0],
                          skiprows=[],
                          names=col_names)

c19_cases

,Summe,Summe Aachen,Summe Todesfälle,Summe genesen,Akute Fälle
Datum,,,,,
2020-03-01,9,4,0,0,9
2020-03-02,10,5,0,0,10
2020-03-03,13,5,0,0,13
2020-03-04,24,8,0,0,24
2020-03-05,33,10,0,0,33
2020-03-06,49,14,0,0,49
2020-03-07,55,15,0,0,55
2020-03-08,57,20,0,0,57
2020-03-09,58,20,0,0,58


## Behandlung fehlender Werte

### Spalte 'Summe Todesfälle'

Am 17.03.2020 wurde das erste Mal in der offiziellen Verlautbarung ein Todesfall erwähnt; für alle Tage vorher wird dieser Spaltenwert zu Null angenommen. Wenn an einem Tag nach dem 17.03. kein Todesfall in der Meldung verzeichnet ist, wird dieser Spaltenwert jeweils fortgeschrieben mit dem letzten offiziell gemeldeten; dadurch kann es zu Sprüngen in der Statistik kommen.

### Spalte 'Summe genesen'

Das erste Mal wurde in der offiziellen Verlautbarung am 10.03.2020 die Zahl der wieder Genesenen erwähnt, bis zum 31.03. erfolgten nur einzelne Meldungen zu sog. "Freitestungen". Erst danach wird die Zahl der Genesenen regelmäßig in den Meldungen der Krisenstäbe erwähnt. Für alle Tage, für die keine Meldung existiert, wurde dieser Spaltenwert jeweils fortgeschrieben mit dem letzten offiziell gemeldeten; dadurch kann es zu Sprüngen in der Statistik kommen.

### Hinzufügen von Tagen, für die keine Fallzahlen vorliegen

Ab dem 01.05.2020 sind die Krisenstäbe von Städteregion und der Stadt Aachen dazu übergegangen, neue Fallzahlen nur noch an normalen Wochentagen zu veröffentlichen; ab Juni wurden nur noch 3x pro Woche die Fallzahlen bekannt gegeben. Damit enthalten die aus dem Excelsheet eingelesenen Zeitreihen tageweise Lücken, insbesondere für die Wochenenden. Diese werden in diesem Schritt aufgefüllt, um die Berechnung gleitender Mittelwerte und Summen später zu vereinfachen.

In [3]:
# Hinzufügen von Tagen, für die keine Fallzahlen vorliegen: Dataframe neu indizieren und fehlende Werte ergänzen

method = config['Kennzahlen']['Impute.Method']

# Fehlende Spaltenwerte auffüllen, indem der jeweils letzte ('ffill') oder nächste ('bfill') gültige Wert 
# fortgeschrieben wird
if method == 'bfill' or method == 'ffill' or method == 'nearest':
    c19_cases = c19_cases.reindex(pd.date_range(c19_cases.index[0], c19_cases.index[-1]), method=method)

c19_cases

,Summe,Summe Aachen,Summe Todesfälle,Summe genesen,Akute Fälle
2020-03-01,9,4,0,0,9
2020-03-02,10,5,0,0,10
2020-03-03,13,5,0,0,13
2020-03-04,24,8,0,0,24
2020-03-05,33,10,0,0,33
2020-03-06,49,14,0,0,49
2020-03-07,55,15,0,0,55
2020-03-08,57,20,0,0,57
2020-03-09,58,20,0,0,58
2020-03-10,61,21,0,6,55


## Korrektur der Spaltenformate
Die Fallzahlen wurden wegen fehlender Werte teilweise als Fließkommazahlen (NaN erfordert ein Float zu Darstellung) importiert, es handelt sich aber in allen Spalten um Ganzzahlen.

In [4]:
# Explizite Umwandlung aller Spalten (außer Index) nach Integer
c19_cases = c19_cases.astype('int32')

## Qualitätskontrolle
Die Summe der Fälle (für die Städteregion) abzüglich der Summe der Todesfälle und abzüglich der Summe der Genesenen muss gleich der Anzahl der akuten Fälle sein. Die resultierende Liste ist leer, wenn alle Zahlen korrekt sind.

In [5]:
c19_cases[c19_cases['Summe'] - c19_cases['Summe Todesfälle'] - c19_cases['Summe genesen'] != c19_cases['Akute Fälle']]

,Summe,Summe Aachen,Summe Todesfälle,Summe genesen,Akute Fälle


## Berechnung zusätzlicher Spalten

- Absolutes und prozentuales Wachstum (oder Rückgang) der aktiven Fälle berechnen
- Anzahl der Neuinfektionen pro Tag (für Stadt Aachen und Städteregion) berechnen
- Anzahl der neuen Todesfälle pro Tag (für Städteregion) berechnen
- Anzahl der neu Genesenen pro Tag (für Städteregion) berechnen
- Gleitendes Mittel (über `Glaettung.Intervall` Tage) für die obigen Kennwerte berechnen
- Summen der Fälle bezogen auf 100.000 Einwohner (für Stadt Aachen und Städteregion) berechnen
- Summe der Neuinfektionen der letzten 7 Tage pro 100.000 Einwohner („7-Tage-Inzidenz“ für Stadt Aachen und Städteregion) berechnen

In [7]:
Einwohner_StädteRegion = int(config['Kennzahlen']['Einwohner.Region'])
Einwohner_StadtAachen = int(config['Kennzahlen']['Einwohner.Aachen'])
Glättungsintervall = int(config['Kennzahlen']['Glaettung.Intervall'])

def change_per_cent(x):
    """
    Evaluate and return the percentage change of the active cases per day by using apply().

    Evaluates difference between first and last array entry, which represents the 'Rolling Window'. 
    Divide by first value and number of days in between.
    
    Parameters
    ----------
    x : []
        Array with case numbers 
    
    Returns
    -------
    float
        Percent change of active cases per day
    """
    if x.size == 1:
        return 0
    else:
        return (x[-1] - x[0]) / x[0] / (x.size - 1) * 100

def diff(x):
    """
    Evaluate new cases / recovered cases / deaths per day by using apply().

    Evaluate difference between first and last array entry, which represents the 'Rolling Window'.
    Divide by number of days in between.

    Parameters
    ----------
    x : []
        Array with case numbers 
    
    Returns
    -------
    float
        Change of cases per day
    """
    if x.size == 1:
        return 0
    else:
        return (x[-1] - x[0]) / (x.size - 1)
    
# Aktive Fälle
c19_cases['Änderung Akute Fälle'] = c19_cases['Akute Fälle'].rolling(2, min_periods=1).apply(diff, raw=True).astype('int32')
c19_cases['Änderung Akute Fälle (MW/{0}T)'.format(str(Glättungsintervall))] = c19_cases['Änderung Akute Fälle'].rolling(Glättungsintervall, center=True).mean()
c19_cases['Änderung Akute Fälle [%]'] = c19_cases['Akute Fälle'].rolling(2, min_periods=1).apply(change_per_cent, raw=True)
c19_cases['Änderung Akute Fälle (MW/{0}T) [%]'.format(str(Glättungsintervall))] = c19_cases['Änderung Akute Fälle [%]'].rolling(Glättungsintervall, center=True).mean()

# Neuinfektionen
c19_cases['Neuinfektionen'] = c19_cases['Summe'].rolling(2, min_periods=1).apply(diff, raw=True).astype('int32')
c19_cases['Neuinfektionen Aachen'] = c19_cases['Summe Aachen'].rolling(2, min_periods=1).apply(diff, raw=True).astype('int32')
c19_cases['Neuinfektionen (MW/{0}T)'.format(str(Glättungsintervall))] = c19_cases['Neuinfektionen'].rolling(Glättungsintervall, center=True).mean()

# Neue Todesfälle
c19_cases['Neue Todesfälle'] = c19_cases['Summe Todesfälle'].rolling(2, min_periods=1).apply(diff, raw=True).astype('int32')
c19_cases['Neue Todesfälle (MW/{0}T)'.format(str(Glättungsintervall))] = c19_cases['Neue Todesfälle'].rolling(Glättungsintervall, center=True).mean()

# Neue Genesene
c19_cases['Neue Genesene'] = c19_cases['Summe genesen'].rolling(2, min_periods=1).apply(diff, raw=True).astype('int32')
c19_cases['Neue Genesene (MW/{0}T)'.format(str(Glättungsintervall))] = c19_cases['Neue Genesene'].rolling(Glättungsintervall, center=True).mean()

# Fälle pro 100.000 Einwohner in den letzten 7 Tagen
c19_cases['Summe pro 100000'] = c19_cases['Summe'] / Einwohner_StädteRegion * 100000
c19_cases['Summe Aachen pro 100000'] = c19_cases['Summe Aachen'] / Einwohner_StadtAachen * 100000
c19_cases['Neuinfektionen 7T/100000 Aachen'] = c19_cases['Neuinfektionen Aachen'].rolling(7, center=False).sum() / Einwohner_StadtAachen * 100000
c19_cases['Neuinfektionen 7T/100000'] = c19_cases['Neuinfektionen'].rolling(7, center=False).sum() / Einwohner_StädteRegion * 100000


## Zwischenergebnis in Excel-Datei speichern

In [8]:
c19_cases.to_excel(config['Kennzahlen']['FileName'], 
                   sheet_name=config['Kennzahlen']['SheetName'],
                   index_label='Datum')

c19_cases.tail(14)

,Summe,Summe Aachen,Summe Todesfälle,Summe genesen,Akute Fälle,Änderung Akute Fälle,Änderung Akute Fälle (MW/7T),Änderung Akute Fälle [%],Änderung Akute Fälle (MW/7T) [%],Neuinfektionen,Neuinfektionen Aachen,Neuinfektionen (MW/7T),Neue Todesfälle,Neue Todesfälle (MW/7T),Neue Genesene,Neue Genesene (MW/7T),Summe pro 100000,Summe Aachen pro 100000,Neuinfektionen 7T/100000 Aachen,Neuinfektionen 7T/100000
2020-10-10,3027,1515,109,2664,254,0,17.714286,0.000000,8.361134,0,0,42.428571,0,0.285714,0,24.428571,543.421672,608.531491,48.200514,38.597839
2020-10-11,3167,1585,109,2723,335,81,18.571429,31.889764,7.725942,140,70,45.857143,0,0.285714,59,27.000000,568.555148,636.648458,58.643959,50.985053
2020-10-12,3167,1585,109,2723,335,0,28.000000,0.000000,9.688790,0,0,57.857143,0,0.285714,0,29.571429,568.555148,636.648458,58.643959,50.985053
2020-10-13,3188,1594,109,2772,307,-28,36.428571,-8.358209,11.320016,21,9,68.857143,0,0.000000,49,32.428571,572.325170,640.263496,59.848972,53.318876
2020-10-14,3257,1626,109,2806,342,35,36.428571,11.400651,11.320016,69,32,68.857143,0,0.000000,34,32.428571,584.712383,653.116967,63.464010,57.627472
2020-10-15,3393,1672,109,2845,439,97,45.285714,28.362573,10.777807,136,46,87.285714,0,0.142857,39,41.857143,609.127761,671.593830,70.694087,72.707558
2020-10-16,3509,1729,109,2891,509,70,45.285714,15.945330,10.777807,116,57,87.285714,0,0.142857,46,41.857143,629.952641,694.489075,85.957584,86.530970
2020-10-17,3509,1729,109,2891,509,0,45.000000,0.000000,11.314519,0,0,89.714286,0,0.142857,0,44.571429,629.952641,694.489075,85.957584,86.530970
2020-10-18,3778,1838,110,3016,652,143,43.285714,28.094303,10.214104,269,109,93.571429,1,0.142857,125,50.142857,678.244822,738.271208,101.622751,109.689673
2020-10-19,3778,1838,110,3016,652,0,45.142857,0.000000,8.598631,0,0,101.428571,0,0.285714,0,56.000000,678.244822,738.271208,101.622751,109.689673
